<a href="https://colab.research.google.com/github/afnf33/emoTale/blob/master/Korean_multisentiment_classifier_KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 구글 드라이브와 연동합니다
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# 필요한 모듈을 설치합니다
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch #원래 ==1.3.1

#SKT에서 공개한 KoBERT 모델을 불러옵니다 
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |███████████████████████████████▌| 834.1MB 1.2MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x6648e000 @  0x7f17eef77615 0x591f47 0x4cc229 0x4cc38b 0x50a51c 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x509918 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x509918 0x50a64d
     |████████████████████████████████| 846.0MB 22kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 9.8MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588513 sha256=13545d772922de120296036e0bdd0bd7dd32a767432644b9fe0a9b92af7b8e7a
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 8.8MB

# 1. 데이터 불러오기, 데이터 전처리

In [17]:
import pandas as pd

sad = pd.read_excel('/content/drive/My Drive/data/tweet_list_슬픔 1~5000.xlsx')
happy = pd.read_excel('/content/drive/My Drive/data/tweet_list_기쁨 labeling_완료.xlsx')
annoy = pd.read_excel('/content/drive/My Drive/data/tweet_list_짜증_완료.xlsx')
fear = pd.read_excel('/content/drive/My Drive/data/tweet_list_공포_완료.xlsx')

sad2 = pd.read_csv('/content/drive/My Drive/data/추가_슬픔.csv')
happy2 = pd.read_csv('/content/drive/My Drive/data/추가_기쁨.csv')
annoy2 = pd.read_csv('/content/drive/My Drive/data/추가_분노.csv')
fear2 = pd.read_csv('/content/drive/My Drive/data/추가_공포1.txt', encoding='utf8')

sad

,Unnamed: 0,raw_text
0,5610,가난한 아버지가 가련한 아들을 껴안고 잠든 밤 마른 이불과 따끈따끈한 요리를 꿈꾸며...
1,2189,가담도 완결나면 유료화되겠지 좀 슬픔
2,1843,가려진시간 뭔데 존나 슬픔 하아 ㅠㅠ 아 시발 진짜 시발 ㅠㅠㅠㅠ
3,497,가리고 싶은 감정이 참 많았어요 뭐라 부르기도 싫고 그냥 그런 것이 없는 사람이 되...
4,2165,가만보면 블루님은 다 귀엽다고 하시는데 본인이 귀여운지는 모르시는 거 같아요 슬픔
...,...,...
2691,7241,오페라의 유령 공연까지 일 남았다 근데 그때면 개강도 머지않아서 슬픔
2692,4421,그런것같아ㅋㅋㅋㅋ 첨에는 목아프더니 갑자기 콧물나오고 장난아니랄까 목 다갈라쟛어 슬픔
2693,4862,그런데 왜 입술을 꾹 물고 슬픔에 물든 눈으로 너를 본다 왜 그렇게 말해
2694,26,언니도 같이 달려줫던 년 좀 슬픔


In [18]:
#전처리를 위한 함수
def preprocessing(data, label):
  import re
  dt = data['raw_text'].copy() #문장만 선택
  dt = dt.dropna() #결측치 제거
  dt = dt.drop_duplicates() #중복 제거
  sentences = dt.tolist()
  new_sent=[]
  for i in range(len(sentences)):
    sent = sentences[i] 
    if type(sent) != str: # 문장 중 str 아닌 것 처리
      sent = str(sent)
    if len(sent) < 2: continue #길이 1 이상인 것만 선택
    sent = re.sub('ㅋㅋ+','ㅋㅋ',sent)
    sent = re.sub('ㅠㅠ+','ㅠㅠ',sent)
    sent = re.sub('ㅇㅇ+','ㅇㅇ',sent) 
    sent = re.sub('ㄷㄷ+','ㄷㄷ',sent) 
    sent = re.sub('ㅎㅎ+','ㅎㅎ',sent)
    sent = re.sub('ㅂㅂ+','ㅂㅂ',sent) 
    sent = re.sub(';;;+',';;',sent)
    sent = re.sub('!!!+','!!',sent)
    sent = re.sub('~+','~',sent)
    sent = re.sub('[?][?][?]+','??',sent) 
    sent = re.sub('[.][.][.]+','...',sent) 
    sent = re.sub('[-=+,#/:$@*\"※&%ㆍ』\\‘|\(\)\[\]\<\>`\'…》]','',sent)
    new_sent.append(sent)
  dt = pd.DataFrame(pd.Series(new_sent), columns=['raw_text'])
  dt['emotion'] = label
  return dt

In [19]:
sad = preprocessing(sad, '슬픔')
sad2 = preprocessing(sad2, '슬픔')
happy = preprocessing(happy, '기쁨')
happy2 = preprocessing(happy2, '기쁨')
annoy = preprocessing(annoy, '분노')
annoy2 = preprocessing(annoy2, '분노')
fear = preprocessing(fear, '공포')
fear2 = preprocessing(fear2, '공포')

for i in [sad, happy, annoy, fear]:
  print('1차 레이블 결과', i['emotion'][0],len(i))
for i in [sad2, happy2, annoy2, fear2]:
  print('2차 레이블 결과', i['emotion'][0],len(i))

print('최소 데이터: 공포 ', len(fear)+len(fear2))

1차 레이블 결과 슬픔 2523
1차 레이블 결과 기쁨 2064
1차 레이블 결과 분노 7862
1차 레이블 결과 공포 1034
2차 레이블 결과 슬픔 2931
2차 레이블 결과 기쁨 3569
2차 레이블 결과 분노 1967
2차 레이블 결과 공포 3280
최소 데이터: 공포  4314


In [41]:
## 데이터 개수 확인 후 학습을 위해 각 감정별 데이터 개수를 동일하게 맞춰줍니다.
sad_3 = sad[:1400]
happy_3 = happy[:800]
annoy_3 = annoy[:2400]

# 각 감정별 키워드 데이터가 약 1000개 씩으로 이루어져 있기 때문에 마지막 키워드에 대한 데이터 1000개를 평가 데이터로 선택
sentence_train = pd.concat([sad_3, happy_3, annoy_3, fear, sad2[:-1000], annoy2[:-1000], happy2[:-1000], fear2[:-1000]], axis=0, ignore_index=True)
sentence_eval = pd.concat([sad2[-1000:], annoy2[-1000:], happy2[-1000:], fear2[-1000:]], axis=0, ignore_index=True)

for i in ['슬픔','기쁨','분노','공포']:
  print('sentence_train',i,len(sentence_train[sentence_train['emotion'] == i]))
print('-------------------------')
for i in ['슬픔','기쁨','분노','공포']:
  print('sentence_eval',i,len(sentence_eval[sentence_eval['emotion'] == i]))

sentence_train 슬픔 3331
sentence_train 기쁨 3369
sentence_train 분노 3367
sentence_train 공포 3314
-------------------------
sentence_eval 슬픔 1000
sentence_eval 기쁨 1000
sentence_eval 분노 1000
sentence_eval 공포 1000


In [40]:
#모델에 입력하기 위해 형식을 맞춰줍니다
def label(x):
  if x=='슬픔': return 0.0
  elif x=='기쁨': return 1.0
  elif x=='분노': return 2.0
  elif x=='공포': return 3.0
  else: return x

sentence_train["emotion"] = sentence_train["emotion"].apply(label)

dtls = [list(sentence_train.iloc[i,:]) for i in range(len(sentence_train))]
dtls[:10] #형식이 통일되었습니다

[['가난한 아버지가 가련한 아들을 껴안고 잠든 밤 마른 이불과 따끈따끈한 요리를 꿈꾸며 잠든 밤 큰 슬픔이 작은 슬픔을 껴안고 잠든 밤 소금 같은 싸락눈이 신문지 갈피를 넘기며 염장을 지르는 지하역의 겨울밤박후기자반고등어',
  0.0],
 ['가담도 완결나면 유료화되겠지 좀 슬픔', 0.0],
 ['가려진시간 뭔데 존나 슬픔 하아 ㅠㅠ 아 시발 진짜 시발 ㅠㅠ', 0.0],
 ['가리고 싶은 감정이 참 많았어요 뭐라 부르기도 싫고 그냥 그런 것이 없는 사람이 되고 싶었어요 제 안에 긍정적이고 밝은 마음만 있기를 바랬나 봐요 그래도 슬픔은 드러내기 어렵지 않았어요 그건 누구에게나 이해받을 수 있다고 생각했지만 미련 유치함 변덕 등은 모두 없었으면 했어요',
  0.0],
 ['가만보면 블루님은 다 귀엽다고 하시는데 본인이 귀여운지는 모르시는 거 같아요 슬픔', 0.0],
 ['가만히있던 슬픔 띠용 ', 0.0],
 ['가발도 구했고 안구도왔구 이제 메이크업만 하면 완성인데 기다리던 분이 한동안 쉬신대슬픔', 0.0],
 ['가방 속에 넣어두었던 돈일반적인 고액은 아니지만 나에게는 고액봉투 사라져서 슬픔 ㅅㅂ 얘야 어디갔ㅇ니 으흑', 0.0],
 ['가버리셨나슬픔에 철푸덕 쓰러짐', 0.0],
 ['가보고싶은 맛집은 참 많은데 인 이상가야 본전 뽑는곳이 너무 많아서 슬픔 ', 0.0]]

# 2. 모델 투입 준비

In [23]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer 
from kobert.pytorch_kobert import get_pytorch_kobert_model 

from transformers import AdamW 
from transformers.optimization import WarmupLinearSchedule 

In [24]:
##GPU 사용 시
device = torch.device("cuda:0")

In [25]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [26]:
#koBERT의 토크나이저를 사용합니다
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [27]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [28]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# 3. 학습

In [29]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1 
log_interval = 200 
learning_rate =  5e-5

In [30]:
# train, validation, test set을 나눠주세요
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(dtls, test_size=0.2, random_state=123)

In [31]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [33]:
#모델을 만들고 GPU 사용 설정을 해줍니다
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [35]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [36]:
#옵티마이저와 손실함수 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

#정확도를 계산하기 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [49]:
#학습 과정
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad() 
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        label = label.long().to(device) 
        out = model(token_ids, valid_length, segment_ids) 
        loss = loss_fn(out, label) 
        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
        optimizer.step() 
        scheduler.step()  # Update learning rate schedule 
        train_acc += calc_accuracy(out, label)  
        if batch_id % log_interval == 0: 
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1))) 
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1))) 
    model.eval() #모델 평가 부분 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 0.10972613841295242 train acc 0.96875

epoch 1 train acc 0.9807477678571429



epoch 1 test acc 0.8313492063492064


In [ ]:
'''
# 차후 사용을 위해 학습된 모델을 저장했습니다
torch.save(model.state_dict(), 'drive/My Drive//kobert_ending_finale.pt')
'''

# 3.5 모델 불러오기

In [ ]:
''' 
# 필요한 모듈을 설치합니다
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch #원래 ==1.3.1

#SKT에서 공개한 KoBERT 모델을 불러옵니다 
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer 
from kobert.pytorch_kobert import get_pytorch_kobert_model 

from transformers import AdamW 
from transformers.optimization import WarmupLinearSchedule 

##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

#koBERT의 토크나이저를 사용합니다
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1 
log_interval = 200 
learning_rate =  5e-5



# 차후 사용시 저장해둔 모델을 불러옵니다
# 필요한 모듈을 다시 설치하고 불러와야 합니다. 또한 class와 def도 재선언 해야 합니다
'''
# #모델을 불러옵니다
# model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# model.load_state_dict(torch.load('drive/My Drive/kobert_ending_finale.pt'))
# model.eval()




BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

#4 평가

In [42]:
##################################################
# 평가용 Test_set을 모델에 입력하기 위해 형식을 맞춰줍니다

sentence_eval["emotion"] = sentence_eval["emotion"].apply(label)

dtls_eval = [list(sentence_eval.iloc[i,:]) for i in range(len(sentence_eval))]

data_test = BERTDataset(dtls_eval, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [50]:
# 해당 데이터에 대해 분류를 시작합니다
model.eval()
answer=[]
train_acc = 0.0
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  max_vals, max_indices = torch.max(out, 1)
  answer.append(max_indices.cpu().clone().numpy())
  test_acc += calc_accuracy(out, label)
print('정답률: ',test_acc / (batch_id+1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



정답률:  0.8313492063492064


In [51]:
# 제출 형식에 맞춰 파일을 저장해줍니다
ls = []
for i in answer:
  ls.extend(i)

pred = pd.DataFrame(ls, columns=['Predicted'])
df = pd.concat([sentence_eval['raw_text'], pred['Predicted'], sentence_eval['emotion']], axis=1) 

def test(x):
  if x==0.0: return '슬픔'
  elif x==1.0: return '기쁨'
  elif x==2.0: return '분노'
  elif x==3.0: return '공포'
  else: return x

df["Predicted"] = df["Predicted"].apply(test)
df["emotion"] = df["emotion"].apply(test)

for i in ['슬픔','기쁨','분노','공포']: 
  print(i, '개수', len(df[df['emotion'] == i]))
  print('예측 개수', len(df[df['emotion'] == i][df['Predicted'] == i]))
  print('정답률',len(df[df['emotion'] == i][df['Predicted'] == i])/len(df[df['emotion'] == i]))

슬픔 개수 1000
예측 개수 963
정답률 0.963
기쁨 개수 1000
예측 개수 845
정답률 0.845
분노 개수 1000
예측 개수 963
정답률 0.963
공포 개수 1000
예측 개수 563
정답률 0.563


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
